In [ ]:
# print any git repository info too (thanks to Travis O'Brien for this code snippet)
import git
try:
    _repo = git.Repo(search_parent_directories=True)
    _git_sha = _repo.head.object.hexsha
    _git_short_sha = _repo.git.rev_parse(_git_sha, short=7)
    _git_branch = _repo.active_branch
    print("On branch {} at rev {}".format(_git_branch,_git_short_sha))
except:
    print("No repository detected.")

In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import holoviews as hv
from holoviews import opts
import geoviews as gv
import geoviews.feature as gf
import matplotlib.pyplot as plt

import geoviews_tools as gt

gv.extension('bokeh')

In [ ]:

ntime = 3
cscratch_path = os.path.join('/', 'global', 'cscratch1', 'sd', 'twhilton')
cscratch_path = '/Users/tim/work/Data/SummenWRF/yatir/'
LHctl_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d03_yatirZ50.nc'),
                               varname='LH',
                               groupname='ctl',
                               timerange=ntime)
LHytr_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d03_yatirZ50.nc'),
                               varname='LH',
                               groupname='yatirZ050',
                               timerange=ntime)
LHctl_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d02_yatirZ50.nc'),
                               varname='LH',
                               groupname='ctl',
                               timerange=ntime)
LHytr_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d02_yatirZ50.nc'),
                               varname='LH',
                               groupname='yatirZ050',
                               timerange=ntime)

ntime = 720
HFXctl_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d03_yatirZ50.nc'),
                                varname='HFX',
                                groupname='ctl',
                                timerange=ntime)
HFXytr_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d03_yatirZ50.nc'),
                                varname='HFX',
                                groupname='yatirZ050',
                                timerange=ntime)
HFXctl_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d02_yatirZ50.nc'),
                                varname='HFX',
                                groupname='ctl',
                                timerange=ntime)
HFXytr_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d02_yatirZ50.nc'),
                                varname='HFX',
                                groupname='yatirZ050',
                                timerange=ntime)

In [ ]:
pad03 = 0.1
pad02 = 1.0
lon_d03 = LHctl_d03.lon.values
lat_d03 = LHctl_d03.lat.values
lon_d02 = LHctl_d02.lon.values
lat_d02 = LHctl_d02.lat.values

In [ ]:
#define some holoviews dimensions
t_dim = hv.Dimension('time', label='date', unit='y/m/d')
hfx_dim = hv.Dimension('HFX', label='sensible heat flux', unit='W m-2')

Plot the data using GeoViews with matplotlib.  The plot shows up (correctly) in the area surrounding [Yatir Forest](https://en.wikipedia.org/wiki/Yatir_Forest).

TODO: why no variable/group labels on this plot?  Perhaps has to do with nesting warning below.

In [ ]:
# LHmaps

TODO: figure out nesting warning below

In [ ]:
# # print(map_LH_ctl_d03) # + map_LH_ytr_d03
# # print(map_LH_ctl_d03.collate())
# print('1-----')
# print(gf.coastline * gv.Dataset(LHytr_d02).to(gv.QuadMesh, groupby='time'))
# print('2-----')
# print(hv.Overlay((gf.coastline, gv.Dataset(LHytr_d02).to(gv.QuadMesh, groupby='time'))))
# # print('3-----')
# # print(LHytr_d02)
# print('4-----')
# print(gv.Dataset(LHytr_d02).to(gv.QuadMesh, groupby='time'))

In [ ]:
# LHmaps = hv.Layout((map_LH_ctl_d02.collate(), 
#             map_LH_ytr_d02.collate(), 
#                 map_LH_ctl_d03.collate(), 
#     map_LH_ytr_d03.collate()),
#                     label='LH (Wm$^{-2}$)').cols(2)

# Yatir parameterization #

In [ ]:
from yatir_land_parameters import format_landparm, format_vegparm
vegparm_str = format_vegparm()
landparm_str = format_landparm()
# print these two strings and paste them into a markdown cell to get a nicely formatted version

#### VEGPARM.TBL ####
|    |   ALBD |      SLMO |      SFEM |      SFZ0 |    THERIN |      SCFX |      SFHC |  LANDCOVER                           |
|---:|-------:|----------:|----------:|----------:|----------:|----------:|----------:|:-------------------------------------|
|  1 |     12 |      0.3  |     0.95  |     50    |       4   |      3.33 |  2.92e+06 | 'Evergreen Needleleaf Forest'        |
|  2 |     12 |      0.5  |     0.95  |     50    |       5   |      1.67 |  2.92e+06 | 'Evergreen Broadleaf Forest'         |
|  3 |     14 |      0.3  |     0.94  |     50    |       4   |      2.86 |  2.5e+06  | 'Deciduous Needleleaf Forest'        |
|  4 |     16 |      0.3  |     0.93  |     50    |       4   |      2.63 |  2.5e+06  | 'Deciduous Broadleaf Forest'         |
|  5 |     13 |      0.3  |     0.97  |     50    |       4   |      2.11 |  4.18e+06 | 'Mixed Forests'                      |
|  6 |     22 |      0.1  |     0.93  |      5    |       3   |      1.56 |  2.08e+06 | 'Closed Shrublands'                  |
|  7 |     20 |      0.15 |     0.95  |      6    |       3   |      2.14 |  2.08e+06 | 'Open Shrublands'                    |
|  8 |     22 |      0.1  |     0.93  |      5    |       3   |      1.56 |  2.08e+06 | 'Woody Savannas'                     |
|  9 |     20 |      0.15 |     0.92  |     15    |       3   |      2    |  2.5e+06  | 'Savannas'                           |
| 10 |     19 |      0.15 |     0.96  |     12    |       3   |      2.37 |  2.08e+06 | 'Grasslands'                         |
| 11 |     14 |      0.42 |     0.95  |     30    |       5.5 |      1.32 |  3.55e+06 | 'Permanent wetlands'                 |
| 12 |     17 |      0.3  |     0.985 |     15    |       4   |      2.71 |  2.5e+06  | 'Croplands'                          |
| 13 |     15 |      0.1  |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Urban and Built-Up'                 |
| 14 |     18 |      0.25 |     0.98  |     14    |       4   |      2.56 |  2.5e+06  | 'cropland/natural vegetation mosaic' |
| 15 |     55 |      0.95 |     0.95  |      0.1  |       5   |      0    |  9e+25    | 'Snow and Ice'                       |
| 16 |     25 |      0.02 |     0.9   |      1    |       2   |      0.81 |  1.2e+06  | 'Barren or Sparsely Vegetated'       |
| 17 |      8 |      1    |     0.98  |      0.01 |       6   |      0    |  9e+25    | 'Water'                              |
| 18 |     15 |      0.5  |     0.93  |     30    |       5   |      2.67 |  9e+25    | 'Wooded Tundra'                      |
| 19 |     15 |      0.5  |     0.92  |     15    |       5   |      2.67 |  9e+25    | 'Mixed Tundra'                       |
| 20 |     12 |      0.02 |     0.97  |     50    |       3   |      1.67 |  1.89e+06 | 'Yatir'                              |
| 21 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 22 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 23 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 24 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 25 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 26 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 27 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 28 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 29 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 30 |     15 |      0.02 |     0.88  |     80    |       3   |      1.67 |  1.89e+06 | 'Unassigned'                         |
| 31 |     10 |      0.1  |     0.97  |     80    |       3   |      1.67 |  1.89e+06 | 'Low Intensity Residential '         |
| 32 |     10 |      0.1  |     0.97  |     80    |       3   |      1.67 |  1.89e+06 | 'High Intensity Residential'         |
| 33 |     10 |      0.1  |     0.97  |     80    |       3   |      1.67 |  1.89e+06 | 'Industrial or Commercial'           |

#### LANDPARM.TBL ####
|    |   SHDFAC |    NROOT |      RS |         RGL |         HS |         SNUP |     MAXALB |      LAIMIN |     LAIMAX |      EMISSMIN |    EMISSMAX |    ALBEDOMIN |    ALBEDOMAX |      Z0MIN |      Z0MAX |      ZTOPV |       ZBOTV |  LANDCOVER'                          |
|---:|---------:|---------:|--------:|------------:|-----------:|-------------:|-----------:|------------:|-----------:|--------------:|------------:|-------------:|-------------:|-----------:|-----------:|-----------:|------------:|:-------------------------------------|
|  1 |     0.7  |        4 |     125 |          30 |      47.35 |        0.08  |         52 |        5    |       6.4  |          0.95 |       0.95  |         0.12 |         0.12 |     0.5    |     0.5    |      17    |        8.5  | 'Evergreen Needleleaf Forest'        |
|  2 |     0.95 |        4 |     150 |          30 |      41.69 |        0.08  |         35 |        3.08 |       6.48 |          0.95 |       0.95  |         0.12 |         0.12 |     0.5    |     0.5    |      35    |        1    | 'Evergreen Broadleaf Forest'         |
|  3 |     0.7  |        4 |     150 |          30 |      47.35 |        0.08  |         54 |        1    |       5.16 |          0.93 |       0.94  |         0.14 |         0.15 |     0.5    |     0.5    |      14    |        7    | 'Deciduous Needleleaf Forest'        |
|  4 |     0.8  |        4 |     100 |          30 |      54.53 |        0.08  |         58 |        1.85 |       3.31 |          0.93 |       0.93  |         0.16 |         0.17 |     0.5    |     0.5    |      20    |       11.5  | 'Deciduous Broadleaf Forest'         |
|  5 |     0.8  |        4 |     125 |          30 |      51.93 |        0.08  |         53 |        2.8  |       5.5  |          0.93 |       0.97  |         0.17 |         0.25 |     0.2    |     0.5    |      18    |       10    | 'Mixed Forests'                      |
|  6 |     0.7  |        3 |     300 |         100 |      42    |        0.03  |         60 |        0.5  |       3.66 |          0.93 |       0.93  |         0.25 |         0.3  |     0.01   |     0.05   |       0.5  |        0.1  | 'Closed Shrublands'                  |
|  7 |     0.7  |        3 |     170 |         100 |      39.18 |        0.035 |         65 |        0.6  |       2.6  |          0.93 |       0.95  |         0.22 |         0.3  |     0.01   |     0.06   |       0.5  |        0.1  | 'Open Shrublands'                    |
|  8 |     0.7  |        3 |     300 |         100 |      42    |        0.03  |         60 |        0.5  |       3.66 |          0.93 |       0.93  |         0.25 |         0.3  |     0.01   |     0.05   |       0.5  |        0.1  | 'Woody Savannas'                     |
|  9 |     0.5  |        3 |      70 |          65 |      54.53 |        0.04  |         50 |        0.5  |       3.66 |          0.92 |       0.92  |         0.2  |         0.2  |     0.15   |     0.15   |       0.5  |        0.1  | 'Savannas'                           |
| 10 |     0.8  |        3 |      40 |         100 |      36.35 |        0.04  |         70 |        0.52 |       2.9  |          0.92 |       0.96  |         0.19 |         0.23 |     0.1    |     0.12   |       0.5  |        0.01 | 'Grasslands'                         |
| 11 |     0.6  |        2 |      70 |          65 |      55.97 |        0.015 |         59 |        1.75 |       5.72 |          0.95 |       0.95  |         0.14 |         0.14 |     0.3    |     0.3    |       0    |        0    | 'Permanent wetlands'                 |
| 12 |     0.8  |        3 |      40 |         100 |      36.25 |        0.04  |         66 |        1.56 |       5.68 |          0.92 |       0.985 |         0.17 |         0.23 |     0.05   |     0.15   |       0.5  |        0.01 | 'Croplands'                          |
| 13 |     0.1  |        1 |     200 |         999 |     999    |        0.04  |         46 |        1    |       1    |          0.88 |       0.88  |         0.15 |         0.15 |     0.5    |     0.5    |       0    |        0    | 'Urban and Built-Up'                 |
| 14 |     0.8  |        3 |      40 |         100 |      36.25 |        0.04  |         68 |        2.29 |       4.29 |          0.92 |       0.98  |         0.18 |         0.23 |     0.05   |     0.14   |       0.5  |        0.01 | 'cropland/natural vegetation mosaic' |
| 15 |     0    |        1 |     999 |         999 |     999    |        0.02  |         82 |        0.01 |       0.01 |          0.95 |       0.95  |         0.55 |         0.7  |     0.001  |     0.001  |       0    |        0    | 'Snow and Ice'                       |
| 16 |     0.01 |        1 |     999 |         999 |     999    |        0.02  |         75 |        0.1  |       0.75 |          0.9  |       0.9   |         0.38 |         0.38 |     0.01   |     0.01   |       0.02 |        0.01 | 'Barren or Sparsely Vegetated'       |
| 17 |     0    |        0 |     100 |          30 |      51.75 |        0.01  |         70 |        0.01 |       0.01 |          0.98 |       0.98  |         0.08 |         0.08 |     0.0001 |     0.0001 |       0    |        0    | 'Water'                              |
| 18 |     0.6  |        3 |     150 |         100 |      42    |        0.025 |         55 |        0.41 |       3.35 |          0.93 |       0.93  |         0.15 |         0.2  |     0.3    |     0.3    |      10    |        0.1  | 'Wooded Tundra'                      |
| 19 |     0.6  |        3 |     150 |         100 |      42    |        0.025 |         60 |        0.41 |       3.35 |          0.92 |       0.92  |         0.15 |         0.2  |     0.15   |     0.15   |       5    |        0.1  | 'Mixed Tundra'                       |
| 20 |     0.56 |        4 |     286 |          30 |      47.35 |        0.08  |         52 |        3    |       3    |          0.97 |       0.97  |         0.12 |         0.12 |     2      |     2      |      10    |        1    | 'Yatir'                              |

# Sensible Heat #

Yatir sensible heat flux in late August 2015 was 368.8 to 434.4 W m$^{-2}$ ([Brugger et al 2018](https://doi.org/10.1007/s10546-018-0371-5). Table 3).  There was a strong diurnal cycle for both sites (forest and desert).  Note the figure units are K m s$^{-1}$.  Figure from [Banerjee et al, 2018]([www.atmos-chem-phys.net/18/10025/2018/).
            <img src="./Banergee_etal_ACP_2018_fig3.pdf" width=900/>

In [ ]:
t_dim = hv.Dimension('time', label='date', unit='y/m/d')
hfx_dim = hv.Dimension('HFX', label='sensible heat flux', unit='W m-2')
area_dim = hv.Dimension('area', label='parameterization', values=['Yatir cells', 'Desert cells'])
WRFrun_dim = hv.Dimension('WRFrun', label='WRF run', values=['Yatir WRF run', 'Control WRF run'])

In [ ]:
dscombined = xr.concat((HFXytr_d03, HFXctl_d03), dim=pd.Index(['Yatir WRF run', 'Control WRF run'], name='WRFrun'))
hvds = hv.Dataset(dscombined, kdims=['time', 'x', 'y', 'WRFrun'], vdims=['HFX', 'lat', 'lon'])

In [ ]:
hfx_yatir = dscombined.where(dscombined.yatir_mask).mean(dim=['x', 'y'])
hfx_desert = dscombined.where(np.logical_not(dscombined.yatir_mask)).mean(dim=['x', 'y'])
hfx = xr.concat((hfx_yatir, hfx_desert), dim=pd.Index(['Yatir cells', 'Desert cells'], name='area'))
hfxds = hv.Dataset(hfx, [WRFrun_dim, area_dim, t_dim], hfx_dim)
#hfxds.to(hv.Curve, [t_dim], [hfx_dim]).opts(width=600).overlay('area')

In [ ]:
from holoviews.plotting.links import RangeToolLink
hfx_curve = hfxds.to(hv.Curve, [t_dim], [hfx_dim]).overlay('area')
hfx_all = hfx_curve.relabel('HFX').opts(width=600, labelled=['y'], toolbar='above')
hfx_zoomed = hfx_curve.opts(width=600, height=200, default_tools=[], toolbar='disable')
RangeToolLink(source=hfx_all.data[('Yatir WRF run',)][('Yatir cells',)], 
              target=hfx_zoomed.data[('Yatir WRF run',)][('Yatir cells',)], 
              axes=['x', 'y'])
layout = (hfx_all + hfx_zoomed).cols(1)
layout.opts(opts.Layout(shared_axes=False, merge_tools=False))

In [ ]:
%matplotlib inline
HFX_masked = dscombined.where(dscombined.yatir_mask)
gv.Dataset(HFX_masked, kdims=['WRFrun', 'time', 'lon', 'lat']).to(hv.QuadMesh, ['lon', 'lat'])[:, ::10, :, :]

Parse Yatir Forest eddy covariance data

In [ ]:
df_ytr = pd.read_csv('/Users/tim/work/Data/Yatir_Forest_Data/EFDC_L2_Flx_ILYat_2015_v03_30m.txt',
                    na_values=[-9999])
df_ytr['time'] = pd.to_datetime(df_ytr['TIMESTAMP_START'], format='%Y%m%d%H%M')
df_ytr.replace()
hv_ytr = hv.Table(df_ytr)
cv_obs = hv_ytr.to.curve('time', 'H', [], label='observed')
cv_obs

Plot HFX for both WRF runs (Control, Yatir) overlaid on a single axes.

In [ ]:
hv.extension('bokeh')
hfx_dim_zoomed = hv.Dimension('HFX_zoomed', label='sensible heat flux', unit='W m-2')
def two_panel_energy_flux_plot(ds, obs, t_str=""):
    """make a plot of Yatir surface energy fluxes"""
    gvds = gv.Dataset(ds, kdims=['WRFrun',  'time', 'x', 'y'])
    WRF_run_labels = ds['WRFrun'].values
    line_styles = ['solid', 'dashed']
    cv = [hv.Curve(gvds.reduce(['x', 'y'], 
                               function=np.nanmean).select(WRFrun=this_run), 
                   t_dim, this_v_dim,
                   label=this_run).opts(opts.Curve(line_dash=this_linestyle, line_width=1))
          for this_run, this_linestyle, this_v_dim in zip(WRF_run_labels, 
                                                          line_styles, 
                                                         [hfx_dim, hfx_dim_zoomed])]
    overlay_opts=opts.Overlay(frame_width=690, 
                              legend_position='right', 
                              ylim=(0, 450),
                             title_format=t_str)
    cv_hfx = hv.Overlay(cv).opts(overlay_opts)
    overlay_opts=opts.Overlay(frame_width=690, legend_position='right', ylim=(350, 450))
    cv_hfx_zoomed = hv.Overlay(cv).opts(overlay_opts)
    hvly = hv.Layout([cv_hfx, cv_hfx_zoomed]).cols(1).opts(norm={'axiswise': True})
    return(hvly, cv_hfx, cv_hfx_zoomed)

In [ ]:
all_d03_cells_time_series, cv_hfx, cv_hfx_zoomed = two_panel_energy_flux_plot(dscombined, cv_obs, "all WRF d03 cells")

[Brugger et al (2018)](https://doi.org/10.1007/s10546-018-0371-5).  Question from Michael - why are the units W m$^{-1}$ and not W m$^{-2}$?
<img src="./Brugger_etal_2018_table3.jpg" width=900/>